# Test of OpenCV + FastAI for video capture

<br>
<br>

### For reference:
<img src='https://i.pinimg.com/originals/fa/bb/70/fabb7087b0cffe30530e7df9ec1d0b88.png' width="400" height="500"></img>
<br>
<br>

In [1]:
# !pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()

# RUN_NAME = '20210116-2147 - arch=xresnet50 - samples=1000 frozen=3 epochs=3 bs=20 res=300 _data=external preuntrained percent=88'
# RUN_NAME = '20210116-2108 - arch=xresnet18 - samples=480 frozen=2 epochs=3 bs=8 res=300 _data=frank_pretrained=false_percent=71'

# DEAD RUN_NAME = '20210118-2100 - arch=xresnet50 - samples=1000 frozen=0 epochs=25 bs=20 res=300 _data=external'
# RUN_NAME = '200927-0730-RESNET101-384px-FALL-ft3f4'
# RUN_NAME = '20210119-0130 - arch=xresnet50 - samples=1000 frozen=0 epochs=25 bs=20 res=300 _data=external'
# RUN_NAME = '20210119-2152 - arch=xresnet50_deep - samples=1000 frozen=0 epochs=25 bs=20 res=300 _data=external'
# RUN_NAME = '20210118-2000 - arch=xresnet50 - samples=1000 frozen=0 epochs=25 bs=20 res=300 _data=external'

# RUN_NAME = '20210121-0017 - arch=xresnet50_deep - samples=3000 frozen=0 epochs=12 bs=20 res=300 _data=external'
# RUN_NAME='20210122-0037 - arch=xresnet50_deep - samples=3000 frozen=1 epochs=40 bs=16 res=360 _data=external'
# RUN_NAME = '20210122-2356 - arch=xresnet50_deep - samples=3000 frozen=1 epochs=40 bs=16 res=360 _data=external'
# RUN_NAME = '20210123-2127 - arch=xresnet50_deep - samples=2000 frozen=1 epochs=22 bs=16 res=360 _data=external'
# RUN_NAME = '20210126-0759 - arch=xresnet50_deep - samples=1000 frozen=1 epochs=30 bs=16 res=360 _data=combined1'
# RUN_NAME = '20210126-1840 - arch=xresnet50_deep - samples=1000 frozen=1 epochs=30 bs=16 res=360 _data=combined1'
# RUN_NAME = '20210127-1717 - arch=xresnet50 - samples=1100 frozen=1 epochs=20 bs=28 res=280 _data=combined2'
# RUN_NAME = '20210127-2251 - arch=xresnet50 - samples=1100 frozen=1 epochs=40 bs=28 res=280 _data=combined2'
# RUN_NAME = '20210128-1251 - arch=xresnet50 - samples=1100 frozen=1 epochs=40 bs=28 res=280 _data=combined2'
# RUN_NAME = '20210128-1704 - arch=xresnet50 - samples=1100 frozen=1 epochs=40 bs=28 res=280 _data=combined2'
RUN_NAME = '20210129-0332 - arch=xresnet50 - samples=620 frozen=1 epochs=40 bs=28 res=280 _data=frank-proc2-L'




PRED_THRESHOLD = 0.85

epoch_num = 23



SAVE_IMAGES_TO_FILE = False  # flag if we want to capture the image to disk

SYMBOL = None

RESOLUTION = 280

PREPROCESS_FRAMES = True
PP_SHOW_PROCESSED_CHANNELS = True


# from fastbook import *
from fastbook import *
from fastai.vision.all import *


from fastai.vision.widgets import *
import fastai.vision
import fastai
from fastai.learner import *
import fastprogress

import numpy as np
import cv2 as cv

import os




## Check for an available GPU

In [2]:
import torch
print('CUDA available: '.ljust(28), torch.cuda.is_available())
print('CUDA device count: '.ljust(28), torch.cuda.device_count())

current_device = torch.cuda.current_device()
print('Current CUDA Device index: '.ljust(28), current_device)
print('Current CUDA Device: '.ljust(28), torch.cuda.get_device_name(current_device))
print()
print(f'fastai version:              {fastai.__version__}')

print(f'cuda version:                {torch.version.cuda}')
print(f'torch version:               {torch.__version__}')
# print(f'python version:              {python_version()}')

CUDA available:              True
CUDA device count:           1
Current CUDA Device index:   0
Current CUDA Device:         GeForce RTX 2070 SUPER

fastai version:              2.2.5
cuda version:                10.1
torch version:               1.7.0


<br>
<br>

## Load the model

In [3]:
if not SAVE_IMAGES_TO_FILE:
    learn_inf = load_learner(f'../models/{RUN_NAME}.pkl', cpu=False)
    load_model(f'models/{RUN_NAME}_{epoch_num}.pth', learn_inf, with_opt=False, opt=Adam)
    print('Model Loaded.')

FileNotFoundError: [Errno 2] No such file or directory: 'models/20210129-0332 - arch=xresnet50 - samples=620 frozen=1 epochs=40 bs=28 res=280 _data=frank-proc2-L_23.pth'

<br>
<br>

## Function definitions

In [ ]:
def setup_camera() -> cv.VideoCapture:
    '''
    Set up the camera source
    '''
#     cap = cv.VideoCapture(0)
    # cap = cv.VideoCapture('http://127.0.0.1:4747/video')
#     cap = cv.VideoCapture('http://10.0.0.67:4747/video')  # S7
    cap = cv.VideoCapture('http://10.0.0.74:4747/video')  # N10

    # cap = cv.VideoCapture('Droidcam')
#     cap = cv.VideoCapture('http://10.0.0.144:4747/video')  # N8
    
    print(f'Autofocus status: {cap.set(cv.CAP_PROP_AUTOFOCUS, 1)}')
    print(f'Manual focus to shortest distance: {cap.set(cv.CAP_PROP_FOCUS, 0)}')

    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error opening video stream or file!")
    
    return cap




def close_camera():
    '''
    Close all capture devices and destroy open capture windows
    '''
    cap.release()
    cv.destroyAllWindows()
    
    


def print_stats():
    '''
    Print some basic stats to stdout
    '''
    
    print(f'Frame shape = {frame.shape}')
    print(f'Total number of Frames = {nframe}')
    print(f'Number of frames processed = {n_proc_frames // PROC_NTH_FRAME}')
    
    
    
# @TODO: Factor out the directory creation logic - slow/redundant    
def save_images(frame, pr_frame):
    '''
    write small and large images to jpeg
    '''
    symbol_dir = SYMBOL
    
    #seperate the directory path
    datadir_small = f'../data/{maindir}-S/{symbol_dir}'
    datadir_large = f'../data/{maindir}-L/{symbol_dir}'

    # build the entire directory structure if it doesn't exist
#     if not os.path.isdir(datadir_small):
#         os.makedirs(datadir_small)
        
    if not os.path.isdir(datadir_large):
        os.makedirs(datadir_large)
    
    # create a directory+filename template
#     ftemplate_small = f'{datadir_small}/{file_prefix}-{n_save_frames}.jpg'
    ftemplate_large = f'{datadir_large}/{file_prefix}-{n_save_frames}.jpg'
    
    # write the image to a file
#     cv.imwrite(ftemplate_small, pr_frame)
#     cv.imwrite(ftemplate_large, frame)
    cv.imwrite(ftemplate_large, pr_frame)
    
    frame = cv.putText(frame, f'Saved Frame {SYMBOL} #{n_save_frames}', 
                   org=(20,40), fontFace=cv.FONT_HERSHEY_PLAIN, 
                   fontScale=2, color=(0,255,255), thickness=2,
                   lineType=cv.LINE_AA) 
    
    # Display the frame with the writing on it
    cv.imshow('What the Camera Sees:',frame)
    
    
    

    


<br>
<br>

## Open the video camera, loop for every Frame, massage the image and make a prediction.  

#### This is the main code cel.  It has 2 purposes. When you hit the spacebar, it will either:
1. Make an ASL alphabet translation or 
2. If SAVE_IMAGES_TO_FILE is True, it will save an image to the data dir 

In [ ]:
from fastai.vision  import *
import torch

PROC_NTH_FRAME = 6  # Skip every N-1 frames - increase this if your computer lags

maindir = 'frank-proc2'  # these are image saving parameters dir/file
file_prefix = 'fproc2'

n_proc_frames, nframe = 0,0  # number of frames
n_save_frames = 0  # number of captured frames for saving to file

# declare here to widen scope
pred, probs, pred_idx = [0], [0], 0
frame, pr_frame = None, None

if SAVE_IMAGES_TO_FILE:
    SYMBOL = input('Enter Symbol of interest: ')[0]
    
cap = setup_camera()
# cap.set(cv.CAP_PROP_FPS, 10)  # Doesn't work?


PP_REMOVE_BACKGROUND = True  # Choose this if you want to try to erase the background
PP_CANNY_EDGE = True  # Choose this to add edge highlighting

if PP_REMOVE_BACKGROUND:
    backSub = cv.createBackgroundSubtractorKNN(history=500, dist2Threshold=300)  # Background Subtractor
    
    ## Create kernels to apply filters
    kernel1 = cv.getStructuringElement(cv.MORPH_ELLIPSE,(1,1))
    kernel2 = cv.getStructuringElement(cv.MORPH_ELLIPSE,(2,2))
    kernel3 = cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))
    kernel5 = cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5))
    # kernel3 = np.ones((3,3),np.uint8)  # kernel of ones for masking
    # kernel5 = np.ones((5,5), np.uint8)
    
    

In [ ]:
    
# "Game" Loop ... for every frame
while(cap.isOpened()):
    nframe += 4
    ret, frame = cap.read()  # Capture frame
    wait_ret = cv.waitKey(1)  # key char value if any

    # Break out of the loop if the user presses 'Q'
    if wait_ret & 0xFF == ord('q'):
        print_stats()
        break
     
    # only process the rest if the capture was successful
    if not ret:
        print("Can't read frame from capture source!")
        break  # break out of loop to let the camera close


    # mirror the image horizontally
    frame = cv.flip(frame, 1)

    # This (background removal) processing must be done every frame, regardless.
    pr_frame = cv.resize(frame[:, 80:-80], (RESOLUTION, RESOLUTION))
    #  pr_frame = np.copy(frame[:,80:-80])

    if PP_REMOVE_BACKGROUND:
        fg_mask = backSub.apply(pr_frame, learningRate=0.005)  # initial background subtraction

    # process only if the space key is hit and
    # only process every PROC_NTH_FRAME frames 
    if (nframe % PROC_NTH_FRAME == 0):  # and wait_ret & 0xFF == ord(' ')  
        n_proc_frames +=1  # num processed frames



        if PREPROCESS_FRAMES:
    
            # remove color (different skin tones, light,etc)
            pr_frame_G = cv.cvtColor(pr_frame, cv.COLOR_BGR2GRAY)  

            if PP_REMOVE_BACKGROUND:

                fg_mask = cv.morphologyEx(fg_mask, cv.MORPH_OPEN, kernel1, iterations=20)  # remove background blocks
                fg_mask = cv.morphologyEx(fg_mask, cv.MORPH_CLOSE, kernel5, iterations=1)  # remove foreground blocks
                fg_mask = cv.medianBlur(fg_mask,5)  # remove salt and pepper
                fg_mask = cv.dilate(fg_mask, kernel5, iterations = 1)  # dilate the mask

                fg_frame = cv.bitwise_and(pr_frame, pr_frame, mask=fg_mask)  # erase the background

                fg_frame_G = cv.cvtColor(fg_frame, cv.COLOR_BGR2GRAY)  # convert to gray
                fg_frame_G = cv.convertScaleAbs(fg_frame_G, alpha=1.2, beta=0)  # increase contrast


                if PP_CANNY_EDGE:  # add canny edge at the end to maintain contrast
                    fg_frame_canny = cv.Canny(fg_frame,50,200)  # Canny edge detection
                    fg_frame_canny = cv.dilate(fg_frame_canny, kernel1)  # increase the line width
        #             fg_frame_canny = cv.morphologyEx(fg_frame_canny, cv.MORPH_GRADIENT, kernel5)
    #                 fg_frame_G = cv.add(fg_frame,fg_frame_canny)  # add the edges back to the masked image.

            ## We now have 3 channels - grey, masked-grey and canny ... we combine them 
            pr_frame[:,:,0] = pr_frame_G  # first channel is grey
            pr_frame[:,:,1] = fg_frame_G  # second channel is with masked background
            pr_frame[:,:,2] = fg_frame_canny  # third channel is canny

#             np.copyto(pr_frame[:,:,0], pr_frame_G)
#             np.copyto(pr_frame[:,:,1], fg_frame_G)
#             np.copyto(pr_frame[:,:,2], fg_frame_canny)


            if PP_SHOW_PROCESSED_CHANNELS:
                cv.imshow('Grey Scale Channel', pr_frame_G)
                cv.imshow('Hidden Background', fg_frame_G)
                cv.imshow('Canny Edge Detection', fg_frame_canny)
        
        
        # Display the frame that we pass through the predictor
        cv.imshow('What the Predictor Sees:', pr_frame)

        if SAVE_IMAGES_TO_FILE :
            if wait_ret & 0xFF == ord(' '):  # only save when pressing the spacebar
                n_save_frames = n_save_frames + 1
                save_images(frame, pr_frame)
            continue  # no need to predict

        # create the prediction
        with learn_inf.no_bar():
            img = PILImage.create(cv.cvtColor(pr_frame, cv.COLOR_BGR2RGB))
            pred,pred_idx,probs = learn_inf.predict(img)
            
           
           
        # write the prediction as text on the frame
        if probs[pred_idx.long()] > PRED_THRESHOLD:
            frame = cv.putText(frame, f'Predict: {pred}, Conf: {probs[pred_idx]:.02f}', 
                               org=(20,40), fontFace=cv.FONT_HERSHEY_PLAIN, 
                               fontScale=2, color=(0,255,255), thickness=2,
                               lineType=cv.LINE_AA) 

        # Display the original frame
        cv.imshow('What the Camera Sees:',frame)

# Release the capture object
close_camera()
# dec_inp.show()

In [ ]:
close_camera()                


<br>
<br>
<br>
<br>
<br>


# The rest of this file is for reference only.  

<br>
<br>
<br>



<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>


# Snippets

```python
# rotate the image
frame = cv.rotate(frame, cv.ROTATE_90_COUNTERCLOCKWISE)
```

In [ ]:
# learn.export(f'../models/{RUN_NAME}.pkl')
# path = Path('../models')
# path.ls(file_exts='.pkl')